In [0]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
import datetime
from functools import partial

In [2]:
data = pd.read_csv('https://www.aspiration.ai/machine-learning/internship/Downloads/stocks/LT.csv')
data

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,LT,EQ,15-May-2017,1740.60,1748.60,1751.55,1736.50,1742.00,1742.40,1743.15,797134,1.389521e+09,52614,529359,66.41
1,LT,EQ,16-May-2017,1742.40,1747.00,1748.25,1727.55,1740.60,1740.70,1736.62,1102926,1.915365e+09,48756,674148,61.12
2,LT,EQ,17-May-2017,1740.70,1743.90,1764.00,1730.50,1754.30,1755.45,1749.36,1668586,2.918955e+09,77513,830692,49.78
3,LT,EQ,18-May-2017,1755.45,1739.65,1750.10,1716.40,1722.10,1722.60,1725.53,1437220,2.479968e+09,47602,915874,63.73
4,LT,EQ,19-May-2017,1722.60,1729.00,1734.00,1697.55,1720.05,1724.55,1718.73,1512094,2.598885e+09,59280,838396,55.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,LT,EQ,07-May-2019,1352.70,1357.40,1374.35,1350.00,1368.50,1366.60,1366.49,2596385,3.547930e+09,80814,1727865,66.55
490,LT,EQ,08-May-2019,1366.60,1363.90,1371.00,1355.00,1362.00,1360.35,1364.13,1601700,2.184919e+09,77662,910828,56.87
491,LT,EQ,09-May-2019,1360.35,1357.00,1368.80,1337.60,1356.00,1356.45,1356.59,2464425,3.343208e+09,97052,1560086,63.30
492,LT,EQ,10-May-2019,1356.45,1359.65,1362.50,1345.10,1356.65,1355.50,1354.49,1466154,1.985897e+09,69058,716794,48.89


In [4]:
filtered_data.head()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,LT,EQ,15-May-2017,1740.60,1748.60,1751.55,1736.50,1742.00,1742.40,1743.15,797134,1.389521e+09,52614,529359,66.41
1,LT,EQ,16-May-2017,1742.40,1747.00,1748.25,1727.55,1740.60,1740.70,1736.62,1102926,1.915365e+09,48756,674148,61.12
2,LT,EQ,17-May-2017,1740.70,1743.90,1764.00,1730.50,1754.30,1755.45,1749.36,1668586,2.918955e+09,77513,830692,49.78
3,LT,EQ,18-May-2017,1755.45,1739.65,1750.10,1716.40,1722.10,1722.60,1725.53,1437220,2.479968e+09,47602,915874,63.73
4,LT,EQ,19-May-2017,1722.60,1729.00,1734.00,1697.55,1720.05,1724.55,1718.73,1512094,2.598885e+09,59280,838396,55.45


In [5]:

filtered_data.tail()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
489,LT,EQ,07-May-2019,1352.70,1357.40,1374.35,1350.0,1368.50,1366.60,1366.49,2596385,3.547930e+09,80814,1727865,66.55
490,LT,EQ,08-May-2019,1366.60,1363.90,1371.00,1355.0,1362.00,1360.35,1364.13,1601700,2.184919e+09,77662,910828,56.87
491,LT,EQ,09-May-2019,1360.35,1357.00,1368.80,1337.6,1356.00,1356.45,1356.59,2464425,3.343208e+09,97052,1560086,63.30
492,LT,EQ,10-May-2019,1356.45,1359.65,1362.50,1345.1,1356.65,1355.50,1354.49,1466154,1.985897e+09,69058,716794,48.89
493,LT,EQ,13-May-2019,1355.50,1345.00,1355.00,1312.0,1318.20,1317.65,1326.93,3422913,4.541968e+09,126334,1686523,49.27


In [6]:
filtered_data.describe()

,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
count,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,4.940000e+02,4.940000e+02,494.000000,4.940000e+02,494.000000
mean,1333.803644,1334.528846,1347.292915,1320.419939,1332.970142,1332.947470,1334.006579,2.309829e+06,3.057544e+09,80733.639676,1.288091e+06,55.459352
std,148.941004,148.383524,149.412145,146.549082,147.629620,147.809173,147.854453,1.381153e+06,1.822511e+09,39185.233377,7.888038e+05,8.703728
min,1117.600000,1120.000000,1126.100000,1113.050000,1118.500000,1117.600000,1120.930000,2.164900e+05,3.023464e+08,6451.000000,1.196130e+05,31.490000
25%,1235.062500,1235.612500,1248.625000,1219.650000,1235.200000,1235.062500,1236.727500,1.471996e+06,1.986064e+09,55263.750000,7.751348e+05,49.860000
50%,1318.250000,1317.950000,1334.775000,1308.000000,1318.975000,1318.025000,1320.680000,1.966720e+06,2.588930e+09,74533.500000,1.101308e+06,55.465000
75%,1371.525000,1373.537500,1386.950000,1360.075000,1371.762500,1371.312500,1373.330000,2.609982e+06,3.535087e+09,93659.500000,1.490642e+06,61.342500
max,1804.900000,1816.000000,1833.950000,1783.500000,1804.350000,1804.900000,1802.250000,1.122784e+07,1.471906e+10,317866.000000,7.378074e+06,79.150000


In [7]:
filtered_data.tail(90)['Close Price'].max()

1441.9

In [8]:
filtered_data.tail(90)['Close Price'].min()

1216.65

In [9]:
filtered_data.tail(90)['Close Price'].mean()

1339.8688888888885

In [10]:
filtered_data['Date'] = pd.to_datetime(filtered_data['Date'])
filtered_data['Date'].dtype

dtype('<M8[ns]')

In [11]:
filtered_data['Date'].max()-filtered_data['Date'].min()


Timedelta('728 days 00:00:00')

In [12]:
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Year'] = pd.DatetimeIndex(data['Date']).year
data['VWAP'] = (data['Close Price'] * data['Total Traded Quantity']).cumsum() / data['Total Traded Quantity'].fillna(0).cumsum()
data_vwap = data[['Month','Year','VWAP']]
group = data_vwap.groupby(['Month','Year'])
group.first()

VWAP
Month Year             
1     2018  1304.498551
      2019  1320.019432
2     2018  1316.817413
      2019  1320.649426
3     2018  1318.742032
      2019  1317.467404
4     2018  1316.047454
      2019  1320.664603
5     2017  1742.400000
      2018  1319.106274
      2019  1322.493637
6     2017  1757.672579
      2018  1322.336568
7     2017  1747.837681
      2018  1320.789454
8     2017  1551.895340
      2018  1318.480134
9     2017  1449.349943
      2018  1318.677904
10    2017  1376.764104
      2018  1318.530114
11    2017  1340.941845
      2018  1312.542263
12    2017  1317.650673
      2018  1316.221853

In [13]:
def avg_price(N):
    return (data['Average Price'].tail(N).sum())/N
print("Average prices for last N days are as follows:")
print("Last 1 week",avg_price(5))
print("Last 2 weeks",avg_price(10))
print("Last 1 month",avg_price(20))
print("Last 3 months",avg_price(60))
print("Last 6 months",avg_price(120))
print("Last 1 year",avg_price(240))
print("Profit/Loss % for N days are as follows:")
def prof_loss(N):
    difference = (data['Close Price'].tail(N).iloc[N-1] - data['Close Price'].tail(N).iloc[0])
    if difference < 0 :
        loss = -(difference)
        loss_percen = (loss/data['Close Price'].tail(N).iloc[N-1])*100
        return loss_percen
    if difference > 0 :
        profit = difference
        profit_percen = (profit/data['Close Price'].tail(N).iloc[N-1])*100
        return profit_percen
print("Loss/Profit percentage for last N days are as follows:")
print("Last 1 week",prof_loss(5))
print("Last 2 weeks",prof_loss(10))
print("Last 1 month",prof_loss(20))
print("Last 3 months",prof_loss(60))
print("Last 6 months",prof_loss(120))
print("Last 1 year",prof_loss(240))

Average prices for last N days are as follows:
Last 1 week 1353.726
Last 2 weeks 1356.383
Last 1 month 1361.2910000000002
Last 3 months 1341.818
Last 6 months 1359.6889999999999
Last 1 year 1331.7889166666666
Profit/Loss % for N days are as follows:
Loss/Profit percentage for last N days are as follows:
Last 1 week 3.7149470648502874
Last 2 weeks 3.077448487838193
Last 1 month 4.090615869160995
Last 3 months 5.399764732668014
Last 6 months 6.135923803741502
Last 1 year 0.18973171934884073


In [14]:
data['Day_Perc_Change'] = data['Close Price'].pct_change().fillna(0)
data

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Month,Year,VWAP,Day_Perc_Change
0,LT,EQ,15-May-2017,1740.60,1748.60,1751.55,1736.50,1742.00,1742.40,1743.15,797134,1.389521e+09,52614,529359,66.41,5,2017,1742.400000,0.000000
1,LT,EQ,16-May-2017,1742.40,1747.00,1748.25,1727.55,1740.60,1740.70,1736.62,1102926,1.915365e+09,48756,674148,61.12,5,2017,1741.413203,-0.000976
2,LT,EQ,17-May-2017,1740.70,1743.90,1764.00,1730.50,1754.30,1755.45,1749.36,1668586,2.918955e+09,77513,830692,49.78,5,2017,1747.976365,0.008474
3,LT,EQ,18-May-2017,1755.45,1739.65,1750.10,1716.40,1722.10,1722.60,1725.53,1437220,2.479968e+09,47602,915874,63.73,5,2017,1740.690629,-0.018713
4,LT,EQ,19-May-2017,1722.60,1729.00,1734.00,1697.55,1720.05,1724.55,1718.73,1512094,2.598885e+09,59280,838396,55.45,5,2017,1736.946183,0.001132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,LT,EQ,07-May-2019,1352.70,1357.40,1374.35,1350.00,1368.50,1366.60,1366.49,2596385,3.547930e+09,80814,1727865,66.55,5,2019,1322.680227,0.010276
490,LT,EQ,08-May-2019,1366.60,1363.90,1371.00,1355.00,1362.00,1360.35,1364.13,1601700,2.184919e+09,77662,910828,56.87,5,2019,1322.733448,-0.004573
491,LT,EQ,09-May-2019,1360.35,1357.00,1368.80,1337.60,1356.00,1356.45,1356.59,2464425,3.343208e+09,97052,1560086,63.30,5,2019,1322.806581,-0.002867
492,LT,EQ,10-May-2019,1356.45,1359.65,1362.50,1345.10,1356.65,1355.50,1354.49,1466154,1.985897e+09,69058,716794,48.89,5,2019,1322.848716,-0.000700


In [15]:
if ((data['Day_Perc_Change'] >= -0.5) & (data['Day_Perc_Change'] <= 0.5)).all():
    data['Trend'] = 'Slight or No change'
if ((data['Day_Perc_Change'] >= 0.5) & (data['Day_Perc_Change'] <= 1)).all():
    data['Trend'] = 'Slight positive'
if ((data['Day_Perc_Change'] <= -0.5) & (data['Day_Perc_Change'] >= -1)).all():
    data['Trend'] = 'Slight negative'
if ((data['Day_Perc_Change'] >= 1) & (data['Day_Perc_Change'] <= 3)).all():
    data['Trend'] = 'Positive' 
if ((data['Day_Perc_Change'] <= -1) & (data['Day_Perc_Change'] >= -3)).all():
    data['Trend'] = 'Negative'
if ((data['Day_Perc_Change'] >= 3) & (data['Day_Perc_Change'] <= 7)).all():
    data['Trend'] = 'Among top gainers'
if ((data['Day_Perc_Change'] <= -3) & (data['Day_Perc_Change'] >= -7)).all():
    data['Trend'] = 'Among top losers'
if (data['Day_Perc_Change'] > 7).all():
    data['Trend'] = 'Bull run' 
if (data['Day_Perc_Change'] < -7).all():
    data['Trend'] = 'Bear drop' 
data

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Month,Year,VWAP,Day_Perc_Change,Trend
0,LT,EQ,15-May-2017,1740.60,1748.60,1751.55,1736.50,1742.00,1742.40,1743.15,797134,1.389521e+09,52614,529359,66.41,5,2017,1742.400000,0.000000,Slight or No change
1,LT,EQ,16-May-2017,1742.40,1747.00,1748.25,1727.55,1740.60,1740.70,1736.62,1102926,1.915365e+09,48756,674148,61.12,5,2017,1741.413203,-0.000976,Slight or No change
2,LT,EQ,17-May-2017,1740.70,1743.90,1764.00,1730.50,1754.30,1755.45,1749.36,1668586,2.918955e+09,77513,830692,49.78,5,2017,1747.976365,0.008474,Slight or No change
3,LT,EQ,18-May-2017,1755.45,1739.65,1750.10,1716.40,1722.10,1722.60,1725.53,1437220,2.479968e+09,47602,915874,63.73,5,2017,1740.690629,-0.018713,Slight or No change
4,LT,EQ,19-May-2017,1722.60,1729.00,1734.00,1697.55,1720.05,1724.55,1718.73,1512094,2.598885e+09,59280,838396,55.45,5,2017,1736.946183,0.001132,Slight or No change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,LT,EQ,07-May-2019,1352.70,1357.40,1374.35,1350.00,1368.50,1366.60,1366.49,2596385,3.547930e+09,80814,1727865,66.55,5,2019,1322.680227,0.010276,Slight or No change
490,LT,EQ,08-May-2019,1366.60,1363.90,1371.00,1355.00,1362.00,1360.35,1364.13,1601700,2.184919e+09,77662,910828,56.87,5,2019,1322.733448,-0.004573,Slight or No change
491,LT,EQ,09-May-2019,1360.35,1357.00,1368.80,1337.60,1356.00,1356.45,1356.59,2464425,3.343208e+09,97052,1560086,63.30,5,2019,1322.806581,-0.002867,Slight or No change
492,LT,EQ,10-May-2019,1356.45,1359.65,1362.50,1345.10,1356.65,1355.50,1354.49,1466154,1.985897e+09,69058,716794,48.89,5,2019,1322.848716,-0.000700,Slight or No change


In [16]:
data.groupby(data.Trend).mean()['Total Traded Quantity']
data.groupby(data.Trend).median()['Total Traded Quantity']

Trend
Slight or No change    1966719.5
Name: Total Traded Quantity, dtype: float64

In [0]:
data.to_csv('Module1Solutions.csv')